# Operator Learning with DeepXDE

In [1]:
import matplotlib.pyplot as plt
import numpy as np
import torch
import torch.utils.data as  dt
import torch.nn as nn

$N$: Number of functions $u(x)$ in training dataset  
$P$: Number fo points inside domain at which $G(u)$ is evaluated (output evaluations)  
$m$: Number of points at which input function is evaluated  

In [2]:

# Load dataset
d = np.load("dataset/antiderivative_unaligned_train.npz", allow_pickle=True)
X_train = (d["X_train0"].astype(np.float32), d["X_train1"].astype(np.float32))
y_train = d["y_train"].astype(np.float32)
d = np.load("dataset/antiderivative_unaligned_test.npz", allow_pickle=True)
X_test = (d["X_test0"].astype(np.float32), d["X_test1"].astype(np.float32))
y_test = d["y_test"].astype(np.float32)

FileNotFoundError: [Errno 2] No such file or directory: 'dataset/antiderivative_unaligned_train.npz'

## Data Generation

Effective data generation requires parallel data generation when training.  
__ID__:  Python string that identifies sample of sample
__train__: Trining data
__validation__: Validation data points  

We will access training and validation samples using `ID`.

In [ ]:
type(X_train[0])

In [ ]:
u = torch.from_numpy(X_train[0])
x  = torch.from_numpy(X_train[1])
v = torch.from_numpy(y_train)

In [ ]:
print(u.shape)#(Number of u Funcs sensor resolution) (150,100)
print(x.shape)#(Number of u funcs,input dimesniom) (100,1)
print(v.shape)# (dataset size, output sensors )(150,100)

In [ ]:
device = torch.device("cuda:0" if torch.has_cuda else "cpu")

In [ ]:
class DataGenerator(dt.Dataset):
    
    def __init__(self,inputs:torch.Tensor,location:torch.Tensor,outputs:torch.Tensor):
        self.input_signals = inputs
        self.collocation_points = location
        self.output_signals = outputs
    
    def __len__(self):
        return len(self.input_signals)
    
    def __getitem__(self, index) -> tuple:
        v = self.input_signals[index,:]
        x = self.collocation_points[index,:]
        u = self.output_signals[index,:]
        return ((v,x),u)

In [ ]:
training_set = DataGenerator(u,x,v)
training_loader = dt.DataLoader(training_set,batch_size = 32,
                                shuffle=True)

In [ ]:
for ((b_in,y_loc),d_out) in training_loader:
    print(b_in.shape)
    print(y_loc.shape)
    print(d_out.shape)
    break

In [ ]:
class NN(nn.Module):
    """Base class for neural network modules"""
    def __init__(self):
        super().__init__()
        self.regulariser = None
    @property
    def num_trainable_parameters(self):
        """Evaluate number of trainable parameters for NN"""
        return sum(v.numel() for v in self.parameters() if v.requires_grad)
#%%
class MLP(NN):
    """Mulilayer perceptron network fully connected"""
    def __init__(self,layer_sizes,
                 activation = nn.ReLU(),
                 kernel_initialiser=nn.init.xavier_normal_,
                 zero_init=nn.init.zeros_):
        super().__init__()
        self.activation = activation       
        self.layers = nn.ModuleList(
            [nn.Linear(l_in,l_out,dtype = torch.float32) 
                for (l_in,l_out) in zip(layer_sizes,layer_sizes[1:])])

        self.apply(self._init_weights)
        
    def _init_weights(self,module:nn.Linear,initialiser = nn.init.xavier_normal_,
                      zero_initialiser = nn.init.zeros_):
        if isinstance(module,nn.Linear):
            initialiser(module.weight)
            zero_initialiser(module.bias)   
    
    def forward(self,inputs):
        x = inputs
        for layer in self.layers[:-1]:
            x = self.activation(layer(x))
        x = self.layers[-1](x)
        
        return self.activation(x)

In [ ]:
class DeepONet(NN):
    """Deep operator network for dataset in the format of Cartesian product.

    Args:
        layer_sizes_branch: A list of integers as the width of a fully connected network,
            or `(dim, f)` where `dim` is the input dimension and `f` is a network
            function. The width of the last layer in the branch and trunk net should be
            equal.
        layer_sizes_trunk (list): A list of integers as the width of a fully connected
            network.
        activation: If `activation` is a ``string``, then the same activation is used in
            both trunk and branch nets. If `activation` is a ``dict``, then the trunk
            net uses the activation `activation["trunk"]`, and the branch net uses
            `activation["branch"]`.
    """

    def __init__(
        self,
        layer_sizes_branch:list,
        layer_sizes_trunk:list,
        *args,**kwargs):
        super().__init__()
        #activation_branch = activation_trunk = activation

        self.branch = MLP(layer_sizes_branch, *args,**kwargs)
        self.trunk = MLP(layer_sizes_trunk, *args,**kwargs)
        self.b = torch.tensor(0.0,requires_grad = True)
    
    def forward(self, inputs):
        v_func:torch.Tensor = inputs[0] # Input signal (batch_size,resolution)
        y_loc:torch.Tensor = inputs[1].swapaxes(2,1) # Collocation points (batch_size,input_dim,num_points)->(b,p,n)
        # Branch net to encode the input function
        v_func = self.branch(v_func)
        # Trunk net to encode the domain of the output function
        y_loc = self.trunk(y_loc).swapaxes(2,1) #Output dim (batch_size,output_layer_dim,)
        # Dot product
        if v_func.shape[-1] != y_loc.shape[1]:
            raise AssertionError(
                "Output sizes of branch net and trunk net do not match.")
        x = torch.einsum("bl,blp->bp", v_func, y_loc)
        # Add bias
        x += self.b
        
        return x

In [ ]:
# Choose a network
m = 100
dim_x = 1
net = DeepONet(
    [m, 40, 40],
    [dim_x, 40, 40])


In [ ]:
import torch.optim as optim

In [52]:
loss_fn = nn.MSELoss()
niter = 10000
opt = optim.Adam(net.parameters(),lr = 1e-3)

In [50]:
2000 % 1000

0

In [55]:
len(training_loader)

313

In [58]:
for epoch in range(5):
    print(f'EPOCH {epoch}')
    
    net.train(True)
    
    running_loss = 0.
    last_loss = 0. 
    
    for i,data in enumerate(training_loader):
        ((u,y),labels) = data
        output = net((u,y[:,None,:]))
        
        opt.zero_grad()
        
        loss = loss_fn(output,labels)
        loss.backward()
        
        opt.step()
        
        running_loss += loss.item()
        
        if i%50 ==49:
            last_loss = running_loss/32
            print(f'Batch :{i} \t Loss\t{last_loss:.5f}')
            running_loss = 0.

EPOCH 0
Batch :49 	 Loss	0.07952
Batch :99 	 Loss	0.09278
Batch :149 	 Loss	0.09814
Batch :199 	 Loss	0.07178
Batch :249 	 Loss	0.08550
Batch :299 	 Loss	0.07971
EPOCH 1
Batch :49 	 Loss	0.08447
Batch :99 	 Loss	0.08626
Batch :149 	 Loss	0.08590
Batch :199 	 Loss	0.08889
Batch :249 	 Loss	0.08770
Batch :299 	 Loss	0.07737
EPOCH 2
Batch :49 	 Loss	0.08509
Batch :99 	 Loss	0.09161
Batch :149 	 Loss	0.08258
Batch :199 	 Loss	0.07760
Batch :249 	 Loss	0.08983
Batch :299 	 Loss	0.08378
EPOCH 3
Batch :49 	 Loss	0.07894
Batch :99 	 Loss	0.08370
Batch :149 	 Loss	0.07284
Batch :199 	 Loss	0.09026
Batch :249 	 Loss	0.09832
Batch :299 	 Loss	0.08609
EPOCH 4
Batch :49 	 Loss	0.09658
Batch :99 	 Loss	0.08765
Batch :149 	 Loss	0.07638
Batch :199 	 Loss	0.08724
Batch :249 	 Loss	0.07935
Batch :299 	 Loss	0.08022


In [ ]:
dddd

In [ ]:

# Define a Model
model = dde.Model(data, net)

# Compile and Train
model.compile("adam", lr=0.001, metrics=["mean l2 relative error"])
losshistory, train_state = model.train(iterations=10000)

# Plot the loss trajectory
dde.utils.plot_loss_history(losshistory)
plt.show()

In [ ]:
net